# Construction Impact with interactive map

### A interactive mapping tool focuses on Road Construction may impact the selected 5 routes in Bay Area CA.

### The Map used multiple market segmentation to get more focused data-driven insight that includes:
- **Year** of construction
- **Road/Lane closure** due to construction
- **Severity** of construction

#### The tool can be used to design scenario testing 

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import geopandas as gpd
from shapely import wkt
from datetime import datetime, timedelta
import datetime as dt
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from os.path import join as join

from ipyleaflet import Map, GeoData, LayersControl, basemaps
import matplotlib.colors as mcolors
import requests
import io
from ipywidgets import VBox
import ipywidgets as widgets
from ipyleaflet import Map, Heatmap, GeoData, LayersControl, basemaps
import gdown

In [2]:
route_url = "https://drive.google.com/uc?export=download&id=1qpx8iktc2wfwCE_2sMv5ODaB3PV4z7HH"

response = requests.get(route_url)
response.raise_for_status()
df_routes = gpd.read_parquet(io.BytesIO(response.content))

In [5]:
# cons_url = "https://drive.google.com/uc?export=download&id=1ROIjuQEbLn74Zs2INz5bfqfe6DZP-Vn2"

# cons_response = requests.get(cons_url)
# cons_response.raise_for_status()
# matched_df = gpd.read_parquet(io.BytesIO(cons_response.content))

url = "https://drive.google.com/uc?id=1ROIjuQEbLn74Zs2INz5bfqfe6DZP-Vn2"

output = "matched_construction_routes.parquet"
gdown.download(url, output, quiet=False)

matched_df = pd.read_parquet(output)

Downloading...
From: https://drive.google.com/uc?id=1ROIjuQEbLn74Zs2INz5bfqfe6DZP-Vn2
To: c:\Users\smomt\OneDrive\Documents\work\Interview\Aurora\Take Home\git\us_road_construction\notebook\matched_construction_routes.parquet
100%|██████████| 217k/217k [00:00<00:00, 1.98MB/s]


In [6]:
# widget for selections
years = sorted(matched_df['year'].dropna().unique())
severity = sorted(matched_df['Severity'].dropna().unique())
road_closure = sorted(matched_df['road_closure'].dropna().unique())


# ToggleButtons
year_toggle = widgets.ToggleButtons(
    options = [('5 Years (2016-2021)', 'all')] + [(str(int(y)), int(y)) for y in years],
    value = 'all',
    description = 'Year:',
    style = {'description_width': 'initial'},
    button_style = 'success', # 'success', 'info', 'warning', 'danger' or ''
)

severity_toggle = widgets.ToggleButtons(
    options = [('All', 'all')] + [(str(int(y)), int(y)) for y in severity],
    value = 'all',
    description = 'Severity:',
    style = {'description_width': 'initial'},
    button_style = 'info', # 'success', 'info', 'warning', 'danger' or ''
)

road_closure_toggle = widgets.ToggleButtons(
    options=[('All', 'all')] + [(y, y) for y in road_closure],
    value='all',
    description='Road/Lane Closure:',
    style={'description_width': 'initial'},
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
)

def create_heatmap(year, severity, road_closure):
    if year == 'all':
        df = matched_df
    else:
        df = matched_df[matched_df['year'] == year]

    if severity_toggle.value == 'all':
        df = df
    else:
        df = df[df['Severity'] == severity]

    if road_closure_toggle.value == 'all':
        df = df
    else:
        df = df[df['road_closure'] == road_closure]
    
    
    heat_df = df[['lat', 'lng', 'duration']].dropna()
    heat_df = heat_df.groupby(['lat', 'lng'], as_index = False)['duration'].sum()
    heat_data = heat_df[['lat', 'lng', 'duration']].values.tolist()

    center = [matched_df['lat'].mean(), matched_df['lng'].mean()]
    m = Map(center=center, zoom=11, basemap=basemaps.CartoDB.DarkMatter, scroll_wheel_zoom=True, layout={'height': '650px'})

    heatmap = Heatmap(locations=heat_data, radius=12, blur=10, min_opacity=0.4, name='Construction Heatmap')
    m.add_layer(heatmap)

    routes_layer = GeoData(
        geo_dataframe=df_routes,
        style_callback=lambda feat: {
            'color': color_map.get(feat['properties']['path_name'], 'white'),
            'weight': 4,
            'opacity': 0.7
        },
        name='Routes',
        hover_style={'fillColor': 'yellow', 'color': 'yellow', 'weight': 6, 'fillOpacity': 0.2},
        tooltip=True,
        tooltip_property='path_name'
    )
    m.add_layer(routes_layer)
    m.add_control(LayersControl())
    return m

map_output = widgets.Output()

def update_map(change):
    with map_output:
        map_output.clear_output()
        selected_year = year_toggle.value
        selected_severity = severity_toggle.value
        selected_road_closure = road_closure_toggle.value
        display(create_heatmap(selected_year, selected_severity, selected_road_closure))

year_toggle.observe(update_map, names='value')
severity_toggle.observe(update_map, names='value')
road_closure_toggle.observe(update_map, names='value')

# Initialization
with map_output:
    display(create_heatmap('all', 'all', 'all'))

display(widgets.VBox([year_toggle, severity_toggle, road_closure_toggle, map_output]))